# Homework 4: Applied ML

We will work with a by-now classic dataset from Robert LaLonde's study "Evaluating the Econometric Evaluations of Training Programs" (1986). The study investigated the effect of a job training program ("National Supported Work Demonstration") on the real earnings of an individual, a couple of years after completion of the program. Your task is to determine the effectiveness of the "treatment" represented by the job training program.

We import all the packages needed here and define plotting functions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nt
import multiprocessing as mp
from scipy.optimize import linear_sum_assignment
from sklearn import linear_model


In [ ]:
def densplot(columns, xlabel, title, axo):
    for i,v in enumerate(columns):
        sns.distplot(v, ax=axo, kde_kws={"label": i})
    axo.set_title(title)
    axo.set_xlabel(xlabel, fontsize=12)
    
def scatplot(xelem, yelem, xlabel, ylabel, title, polyfit=None):
    plt.scatter(xelem, yelem)
    if polyfit:
        plt.plot(np.unique(xelem), np.poly1d(np.polyfit(xelem, yelem, polyfit))(np.unique(xelem)), 'C2')
    plt.title(title)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.show()

def boplot(data, title, xlabel, violin, axo): #violin=True means a quantil plot
    if violin:
        sns.violinplot(data=data, ax=axo)
    else:
        sns.boxplot(data=data, ax=axo)
    axo.set_title(title)
    axo.set_xlabel(xlabel, fontsize=12)

def valCountBar(data, title, xlabel, ylabel, axo, color):
    data.value_counts().plot(kind='bar', ax=axo, color=color)
    axo.set_title(title)
    axo.set_xlabel(xlabel, fontsize=12)
    axo.set_ylabel(ylabel, fontsize=12)

We start by importing the data.

#### Dataset description

- `treat`: 1 if the subject participated in the job training program, 0 otherwise
- `age`: the subject's age
- `educ`: years of education
- `race`: categorical variable with three possible values: Black, Hispanic, or White
- `married`: 1 if the subject was married at the time of the training program, 0 otherwise
- `nodegree`: 1 if the subject has earned no school degree, 0 otherwise
- `re74`: real earnings in 1974 (pre-treatment)
- `re75`: real earnings in 1975 (pre-treatment)
- `re78`: real earnings in 1978 (outcome)

In [ ]:
data = pd.read_csv('lalonde.csv', index_col=0)
data.head()

To match with the description and for simplicity in futur steps, we create the "race" and "white" columns:
(we first check that there are no people that are both black and hispanic in the dataset)

In [ ]:
print("Number of black hispanic people: ", len(data.loc[(data['black']==1) & (data['hispan'] == 1)]))
data["race"] = "white"
data["white"] = 0
data.loc[data['black'] == 1,'race'] = "black"
data.loc[data['hispan'] == 1,'race'] = "hispanic"
data.loc[(data['black'] == 0) & (data['hispan'] == 0),'white'] = 1
data.head(10)

### 1. A naive analysis

In this part, we compare the mean and distribution of the outcome variable (`re78`) between the two groups (control and test)

In [ ]:
grouped = data.groupby('treat')
grouped0 = grouped.get_group(0)
grouped1 = grouped.get_group(1)

In [ ]:
print("The people that didn't follow the programm have an average salary of", grouped0['re78'].mean(), " when the ones that followed are at ", grouped1['re78'].mean())

In [ ]:
fig, ax1 = plt.subplots(nrows=1, ncols=1, sharey=True, figsize=(10,5))
densplot([grouped0['re78'],grouped1['re78']], "treat", "Comparison of distributions", ax1)
plt.show()

**What might a naive "researcher" conclude from this superficial analysis?**
A very naive observer that only compares the average salaries could think that the program has a negative effect, as the average salaray is lower for those who followed the program.
A little less naive observer would think that the two distributions are pretty similar even though there seems to be more people that didn't follow the program with an income between 1500 and 2000.


In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(10,5))
boplot([grouped0['re78'],grouped1['re78']],"Treat comparison boxplots", "treat", False, ax1)
boplot([grouped0['re78'],grouped1['re78']],"Treat comparison violin plots", "treat", True, ax2)
plt.show()

The boxplot and violin plot (combination of boxplot and kernel density estimate) tend to show the same as we observed previously. Nevertheless, they also show that the group that followed the program contains more outliers, or in other words, more people with (much) higher incomes.
*A naive conclusion would be to say that the program is not very efficient except for a few people that have a very high income*

### 2. A closer look at the data

For each feature in the dataset, we compare its distribution in the treated group with its distribution in the control group. 
Since each feature is pretty different, we cannot plot everything at the same time and we will do each feature at a time.

#### Age

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(10,5))
densplot([grouped0["age"],grouped1["age"]], "age", "Age Distribution (density)", ax1)
boplot([grouped0["age"],grouped1["age"]],"Age Distribution (boxplot)", "treat", False, ax2)
plt.show()

We observe that even if the median is similar in both datasets according to the age feature, the dataset of people that didn't follow the program (treat=0) contains more older people while the people that followed the program where concentrated between 20 and 30 years old.

*A first observation that we can do, is that the income is often related to the age, as more expericence usually mean a better income. Thus, the fact that the two datasets have different age distribution should be considered in the comparison*

#### Education

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(15,5))
densplot([grouped0["educ"],grouped1["educ"]], "years of education", "Education duration distribution (density)", ax1)
boplot([grouped0["educ"],grouped1["educ"]],"Education duration distribution (boxplot)", "treat", False, ax2)
plt.show()

We see that in terms of education duration, the values are more sparsed for people that didn't follow the program with outliers in both direction (with few or many years of education). Again the median is similar in both datasets but we observe that they are more people that followed the program that received between 10 and 12 years of education while the other dataset is more sparsed with a peak at 13 years of education.


#### Race

We now observe the "race" feature that we built from the "black" and "hispan" features in the beginning

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(13,5))
valCountBar(grouped0['race'], "Race counts for people that didn't follow the program", "race", "count", ax1, 'b')
valCountBar(grouped1['race'], "Race counts for people that followed the program", "race", "count", ax2, 'r')
plt.show()

In this case, we do not need more plots to see that the people that didn't follow the program were mostly white while the other dataset almost only contains black people.
*Again, it is well known that, sadly, the average income for black people is usually lower than for white people which can also interfer with the results of the study*

#### Married

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(13,5))
valCountBar(grouped0['married'], "Married counts for people that didn't follow the program", "married (1) or not married (0)", "count", ax1, 'b')
valCountBar(grouped1['married'], "Married counts for people that followed the program", "married (1) or not married (0)", "count", ax2, 'r')
plt.show()

We clearly observe that most of the people that followed the program were single when the married against single counts are balanced in the first dataset.

*The fact that someone is married or not can also have an influence on its income and should be taken into account*

#### Degree

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(13,5))
valCountBar(grouped0['nodegree'], "Counts for people with/without a degree that didn't follow the program", "No degree (1) or degree (0)", "count", ax1, 'b')
valCountBar(grouped1['nodegree'], "Counts for people with/without a degree that followed the program", "No degree (1) or degree (0)", "count", ax2, 'r')
plt.show()

In this case, both datasets are consistent and about two-third of the people do not have a degree in each.

#### real earnings in 1974 (pre-treatment)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(15,5))
densplot([grouped0["re74"],grouped1["re74"]], "real earnings in 1974", "Comparison of distributions for real earnings in 1974", ax1)
boplot([grouped0["re74"],grouped1["re74"]],"Comparison with boxplots for real earnings in 1974", "treat", False, ax2)
plt.show()

We observe that both datasets contain a lot of people with no income at this date. We also notice that more people that didn't participate in the program already had a salary at this date which is less the case for people that participated.

#### real earnings in 1975 (pre-treatment)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(15,5))
densplot([grouped0["re75"],grouped1["re75"]], "real earnings in 1975", "Comparison of distributions for real earnings in 1975", ax1)
boplot([grouped0["re75"],grouped1["re75"]],"Comparison with boxplots for real earnings in 1975", "treat", False, ax2)
plt.show()

We observe that the income decreased for a lot of people in the first dataset, when it increases for some in the second dataset.

#### Conclusion

We discuss here our main observations and how they relate to our naive observation.
- the dataset of people that didn't followed the program contains more people that are above 30 years old
- most of the people that followed the program were single, which is not the case in the other dataset
- more people already had an income before 1975 in the people that didn't participate in the program

From these observations, we can remark that even though the median values are often similar in both datasets, the distributions are different. Moreover, this is particularly the case for the age and race features which can both have a high impact on the income of a person.


### 3. A propensity score model

We use logistic regression to estimate propensity scores for all points in the dataset.

In [ ]:
logistic = linear_model.LogisticRegression()
feature_cols = ['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75',]
X = data[feature_cols]
y = data['treat']

propensity = logistic.fit(X, y)
pscore = propensity.predict_proba(X)[:,1] # The predicted propensities by the model
print(pscore[:10])

data['propensity'] = pscore

### 4. Balancing the dataset via matching

We use the propensity scores to match each data point from the treated group with exactly one data point from the control group, while ensuring that each data point from the control group is matched with at most one data point from the treated group.

We start our matching process by creating the matrix of the costs (using the absolute difference between the propensity scores as a cost function) of all the possible assignments. We then use this matrix to solve the linear sum assignment problem (The linear sum assignment problem is also known as minimum weight matching in bipartite graphs).


In [ ]:
group2 = data.groupby('treat')
group20 = group2.get_group(0).reset_index()
group21 = group2.get_group(1).reset_index()

In [ ]:
def col(colone, coltwo):
    return np.array([abs(colone - n) for n in coltwo])

costs = np.array(col(group21['propensity'], group20['propensity']))

print("Computing optimal assigment...", end="")
id_n1, id_n2 = linear_sum_assignment(costs)
sol_costs = costs[id_n1, id_n2]

print("Done: cost of solution = %f" % sol_costs.sum())

match = group21.join(group20.loc[id_n1].reset_index(drop=True), rsuffix="_notreat")
len(match)

After the matching, we compare the outcomes (`re78`) between the two groups (treated and control)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(15,5))
densplot([match["re78"], match["re78_notreat"]], "real earnings in 1978", "Comparison of distributions for real earnings in 1978", ax1)
boplot([match["re78"], match["re78_notreat"]],"Comparison with boxplots for real earnings in 1978", "treat", False, ax2)
plt.show()

As we observed in the beginning, the two datasets have similar distributions even though the group of people that didn't follow the program are still more numerous to have big incomes.

We compare again the feature-value distributions between the two groups but now only for the matched subjects.
In this case, we start with all the features that are now well-balanced between the two datasets.

It is clearly the case for: **married, no degree, study duration, real earnings in 1974 (pre-treatment), real earnings in 1975 (pre-treatment)**

In [ ]:
fig, ((ax1, ax2), (ax3, ax4),(ax5, ax6),(ax7, ax8)) = plt.subplots(nrows=4, ncols=2, sharey=False, figsize=(15,20))
valCountBar(match['married_notreat'], "Married counts for people that didn't follow the program", "Married (1) or not (0)", "count", ax1, 'b')
valCountBar(match['married'], "Married counts for people that followed the program", "Married (1) or not (0)", "count", ax2, 'r')
valCountBar(match['nodegree_notreat'], "Counts for people with/without a degree that didn't follow the program", "No degree (1) or degree (0)", "count", ax3, 'b')
valCountBar(match['nodegree'], "Counts for people with/without a degree that followed the program", "No degree (1) or degree (0)", "count", ax4, 'r')
densplot([match["educ_notreat"],match["educ"]],"Comparison with boxplots for education duration", "treat", ax5)
densplot([match["re74_notreat"],match["re74"]],"Comparison with boxplots for real earnings in 1974", "treat", ax6)
densplot([match["re75_notreat"],match["re75"]],"Comparison with boxplots for real earnings in 1975", "treat", ax7)
fig.tight_layout()
plt.show()

The remaining features for which the balance is still not completely attaigned are:

#### age

We observe that the remaining people are mostly between 17 and 27 years old in the non-program group and between 20 and 28 in the second one. We will come back to this feature if our next improved matching is still not good enough.

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(10,5))
densplot([match["age_notreat"], match["age"]], "treat", "Comparison with boxplots for age", ax1)
boplot([match["age_notreat"], match["age"]],"Comparison with boxplots for age", "treat", False, ax2)
plt.show()

#### Race

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(13,5))
valCountBar(match['race_notreat'], "Race counts for people that didn't follow the program", "race", "count", ax1, 'b')
valCountBar(match['race'], "Race counts for people that followed the program", "race", "count", ax2, 'r')
plt.show()

**Are you closer to being able to draw valid conclusions now than you were before?**
We are closer than before as the dataset is well-balanced for multiple features. Nevertheless, they are still three features for which the balance is not perfect. The worst one seems to be the race with a dataset that almost only contains black people (treat = 1) when the other is more internally balanced. 

### 5. Balancing the groups further

Since the race feature seems to be the most problematic one, we first try to solve this issue by keeping only the matched people with a matching race. Since the matching based on the propensity score and done using a linear sum assignement is optimal, removing matched people seems to be a better solution that trying to redo the match while changing the distance function by giving a high weight to the race feature.

In [ ]:
matchRace = match.loc[match['race'] == match['race_notreat']]
print("There are " , len(matchRace), " out of ", len(match), " that are matched with someone of the same race")

We check the effect of this operation on the other variables:

In [ ]:
fig, ((ax1, ax2), (ax3, ax4),(ax5, ax6),(ax7, ax8)) = plt.subplots(nrows=4, ncols=2, sharey=False, figsize=(15,20))
valCountBar(matchRace['married_notreat'], "Married counts for people that didn't follow the program", "Married (1) or not (0)", "count", ax1, 'b')
valCountBar(matchRace['married'], "Married counts for people that followed the program", "Married (1) or not (0)", "count", ax2, 'r')
valCountBar(matchRace['nodegree_notreat'], "Counts for people with/without a degree that didn't follow the program", "No degree (1) or degree (0)", "count", ax3, 'b')
valCountBar(matchRace['nodegree'], "Counts for people with/without a degree that followed the program", "No degree (1) or degree (0)", "count", ax4, 'r')
densplot([matchRace["re74_notreat"],matchRace["re74"]],"treat", "Comparison with boxplots for real earnings in 1974", ax5)
densplot([matchRace["re75_notreat"],matchRace["re75"]],"treat", "Comparison with boxplots for real earnings in 1975", ax6)
densplot([matchRace["educ_notreat"],matchRace["educ"]],"treat", "Comparison with boxplots for education duration", ax7)
densplot([matchRace["age_notreat"], matchRace["age"]],"treat", "Comparison with boxplots for age", ax8)
fig.tight_layout()
plt.show()

We still observe that the age distribution is not perfectly balanced. But, in the next cell we observe that 53 out of 86 matched people have an age difference smaller than 10 years. We will discuss this more in the next section.

In [ ]:
ageStd1 = int(pd.DataFrame.std(matchRace['age']))
ageStd2 = int(pd.DataFrame.std(matchRace['age_notreat']))
ageStdMax = max(ageStd1, ageStd2)
print("The standard deviation of ages are ", ageStd1, " and ", ageStd2)
matchAge = matchRace.loc[abs(matchRace['age'] - matchRace['age_notreat']) < ageStdMax]
print("There are " , len(matchAge), " out of ", len(matchRace), " that are matched with someone that have similar age ")

### 6. A less naive analysis

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(10,5))
densplot([matchRace['re78_notreat'],matchRace['re78']],"treat", "Treat comparison boxplots matched on Race", ax1)
boplot([matchRace['re78_notreat'],matchRace['re78']],"Treat comparison boxplots matched on Race", "treat", False, ax2)
plt.show()
meanNoTreat = matchRace['re78_notreat'].mean()
meanTreat = matchRace['re78'].mean()
print("Now the average salary for people that didn't follow the program is ", meanNoTreat, " and is ", meanTreat, " for those who followed the program")
diff = (meanTreat-meanNoTreat)*100/meanTreat
print("People that followed the proram have salaries ", diff, " higher." )

We can further check if matching on age change this result:

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(10,5))
densplot([matchAge['re78_notreat'],matchAge['re78']],"treat", "Treat comparison boxplots matched on Race", ax1)
boplot([matchAge['re78_notreat'],matchAge['re78']],"Treat comparison boxplots matched on Race", "treat", False, ax2)
plt.show()
meanNoTreat = matchAge['re78_notreat'].mean()
meanTreat = matchAge['re78'].mean()
print("Now the average salary for people that didn't follow the program is ", meanNoTreat, " and is ", meanTreat, " for those who followed the program")
diff = (meanTreat-meanNoTreat)*100/meanTreat
print("People that followed the proram have salaries ", diff, " higher." )

We observe that in this case, the average salary decreases in both datasets but the difference between the two remains almost the same.

## Final Conclusion

From our analysis, we can conclude that the program has a little but positive effect on the salaries of the participants.